### Selected Word Group In Youtube Sentence

In [14]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [15]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File 28 Word Twogram"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "The_Most_File_28_Word_Twogram"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 10  # 6

# youtube sentence condition
sent_time_start = False  # True, False for activating start time condition
sent_start_time = 700  # sn
random_sample = True  # True, False

In [16]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [17]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [18]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [19]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    
    return  df_word_count

In [20]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [21]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [22]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [23]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [24]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [25]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,77.591,79.814,nasıl olması gerektiğiyle ilgili,Tm4hNlWWa_8
1,121.837,123.980,elim boş ne güzel gidiyordum,Kp5M0OHZqhc
2,4551.091,4552.427,aç aç,fzJ_4cmQRKA
3,4978.053,4979.267,hı hı,dcrF0fcizRA
4,2994.186,2996.598,sonsuza kadar kurtulmam gerekiyor,Kjw9SWuqcZI
...,...,...,...,...
3934202,78.670,90.220,bunları keseceğim daha ayrıntılı görünmesi içi...,VOjapojSyEQ
3934203,295.679,301.823,yine aynı bölgede soğuk olan sıcaklığı gece sa...,GyOoAw38KnE
3934204,1754.359,1756.295,tombul ne ha tombul ne,rBmOKZYKOhE
3934205,4198.179,4201.092,vallahi bak krallar bile yiyemez böyle yemeği ha,7oJN2fCL7ak


In [26]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,twogram
0,bir şey
1,değil mi
2,ben de
3,var mı
4,ne kadar
5,bu kadar
6,bu da
7,sen de
8,bu çok
9,ne var


In [ ]:
# Option 
# Skip if it doesn't need
remove_videoid_list = list(set(df_word_group_select["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [27]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [28]:
len(search_list)

30

In [29]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,evet ama,2519.340,2523.840,tamam sakin ol tamam canım yaşadığımız çok ağı...,F1EQ6xghU9E
1,evet ama,1183.939,1187.651,evet ama bu çarpma olayı çok daha başka bir şey,VGntaV10208
2,evet ama,1093.923,1098.003,evet dramı öğrendik evet haksızlığı öğrendik e...,k7YWcr0GwLU
3,evet ama,4003.699,4005.494,evet ama zili bulamadım,6HX5dmPKqWQ
4,evet ama,2893.614,2895.670,evet ama nasıl nasıl olacak,0EmRyNfzWKM
...,...,...,...,...,...
295,bu ne,730.312,731.548,bu ne içindi,SRBkkbE7Uyg
296,bu ne,6889.740,6893.880,görüyor musun yaptığını ya bu ne demek oluyor ...,IvcoZk5ZsyI
297,bu ne,584.851,592.168,iki tane oğlan verdim bütün gün yerlerdesin bu...,tlPXUeUI4zc
298,bu ne,459.120,461.480,bu ne diyor hiç anlamıyorum ama çok iyi diyor ha,oE9qrxR7DpI


In [30]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,evet ama,2523.141724,2523.840000,tamam sakin ol tamam canım yaşadığımız çok ağı...,F1EQ6xghU9E
1,evet ama,1183.939000,1184.649809,evet ama bu çarpma olayı çok daha başka bir şey,VGntaV10208
2,evet ama,1097.310170,1098.003000,evet dramı öğrendik evet haksızlığı öğrendik e...,k7YWcr0GwLU
3,evet ama,4003.699000,4004.401391,evet ama zili bulamadım,6HX5dmPKqWQ
4,evet ama,2893.614000,2894.299333,evet ama nasıl nasıl olacak,0EmRyNfzWKM
...,...,...,...,...,...
295,bu ne,730.312000,730.930000,bu ne içindi,SRBkkbE7Uyg
296,bu ne,6891.476129,6891.943548,görüyor musun yaptığını ya bu ne demek oluyor ...,IvcoZk5ZsyI
297,bu ne,587.876298,588.368788,iki tane oğlan verdim bütün gün yerlerdesin bu...,tlPXUeUI4zc
298,bu ne,459.120000,459.415000,bu ne diyor hiç anlamıyorum ama çok iyi diyor ha,oE9qrxR7DpI


In [31]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,evet ama,2522.541724,2524.440000,tamam sakin ol tamam canım yaşadığımız çok ağı...,F1EQ6xghU9E
1,evet ama,1183.339000,1185.249809,evet ama bu çarpma olayı çok daha başka bir şey,VGntaV10208
2,evet ama,1096.710170,1098.603000,evet dramı öğrendik evet haksızlığı öğrendik e...,k7YWcr0GwLU
3,evet ama,4003.099000,4005.001391,evet ama zili bulamadım,6HX5dmPKqWQ
4,evet ama,2893.014000,2894.899333,evet ama nasıl nasıl olacak,0EmRyNfzWKM
...,...,...,...,...,...
295,bu ne,729.712000,731.530000,bu ne içindi,SRBkkbE7Uyg
296,bu ne,6890.876129,6892.543548,görüyor musun yaptığını ya bu ne demek oluyor ...,IvcoZk5ZsyI
297,bu ne,587.276298,588.968788,iki tane oğlan verdim bütün gün yerlerdesin bu...,tlPXUeUI4zc
298,bu ne,458.520000,460.015000,bu ne diyor hiç anlamıyorum ama çok iyi diyor ha,oE9qrxR7DpI


In [32]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,evet ama,2523,2524,tamam sakin ol tamam canım yaşadığımız çok ağı...,F1EQ6xghU9E
1,evet ama,1183,1185,evet ama bu çarpma olayı çok daha başka bir şey,VGntaV10208
2,evet ama,1097,1099,evet dramı öğrendik evet haksızlığı öğrendik e...,k7YWcr0GwLU
3,evet ama,4003,4005,evet ama zili bulamadım,6HX5dmPKqWQ
4,evet ama,2893,2895,evet ama nasıl nasıl olacak,0EmRyNfzWKM
...,...,...,...,...,...
295,bu ne,730,732,bu ne içindi,SRBkkbE7Uyg
296,bu ne,6891,6893,görüyor musun yaptığını ya bu ne demek oluyor ...,IvcoZk5ZsyI
297,bu ne,587,589,iki tane oğlan verdim bütün gün yerlerdesin bu...,tlPXUeUI4zc
298,bu ne,459,460,bu ne diyor hiç anlamıyorum ama çok iyi diyor ha,oE9qrxR7DpI


In [33]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,evet ama,2523,2524,tamam sakin ol tamam canım yaşadığımız çok ağı...,F1EQ6xghU9E,https://www.youtube.com/watch?v=F1EQ6xghU9E&t=...
1,evet ama,1183,1185,evet ama bu çarpma olayı çok daha başka bir şey,VGntaV10208,https://www.youtube.com/watch?v=VGntaV10208&t=...
2,evet ama,1097,1099,evet dramı öğrendik evet haksızlığı öğrendik e...,k7YWcr0GwLU,https://www.youtube.com/watch?v=k7YWcr0GwLU&t=...
3,evet ama,4003,4005,evet ama zili bulamadım,6HX5dmPKqWQ,https://www.youtube.com/watch?v=6HX5dmPKqWQ&t=...
4,evet ama,2893,2895,evet ama nasıl nasıl olacak,0EmRyNfzWKM,https://www.youtube.com/watch?v=0EmRyNfzWKM&t=...
...,...,...,...,...,...,...
295,bu ne,730,732,bu ne içindi,SRBkkbE7Uyg,https://www.youtube.com/watch?v=SRBkkbE7Uyg&t=...
296,bu ne,6891,6893,görüyor musun yaptığını ya bu ne demek oluyor ...,IvcoZk5ZsyI,https://www.youtube.com/watch?v=IvcoZk5ZsyI&t=...
297,bu ne,587,589,iki tane oğlan verdim bütün gün yerlerdesin bu...,tlPXUeUI4zc,https://www.youtube.com/watch?v=tlPXUeUI4zc&t=...
298,bu ne,459,460,bu ne diyor hiç anlamıyorum ama çok iyi diyor ha,oE9qrxR7DpI,https://www.youtube.com/watch?v=oE9qrxR7DpI&t=...


In [34]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

30

In [35]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [36]:
search_all_set.difference(search_result_set)

set()

In [37]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,için,5
2,ne,5
1,bu,5
3,kadar,3
4,ben,3
5,çok,3
6,bir,3
7,da,3
8,daha,3
9,de,3


In [38]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [39]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [40]:
set_word.difference(set_count)

{'adam',
 'al',
 'anne',
 'artık',
 'asla',
 'aslında',
 'aynı',
 'baba',
 'bak',
 'bakalım',
 'başka',
 'belki',
 'bence',
 'beni',
 'benimle',
 'bile',
 'biliyor',
 'biliyorsun',
 'biliyorum',
 'bilmiyorum',
 'biraz',
 'biri',
 'birkaç',
 'birlikte',
 'biz',
 'bize',
 'bizi',
 'bizim',
 'bugün',
 'buna',
 'bunu',
 'burada',
 'buraya',
 'böyle',
 'bütün',
 'büyük',
 'dakika',
 'demek',
 'devam',
 'değilim',
 'diye',
 'dostum',
 'doğru',
 'dur',
 'ederim',
 'efendim',
 'en',
 'et',
 'eğer',
 'fazla',
 'gece',
 'gel',
 'geldi',
 'geliyor',
 'gerek',
 'geri',
 'gerçek',
 'gerçekten',
 'gibi',
 'git',
 'göre',
 'gün',
 'güzel',
 'hadi',
 'hakkında',
 'hala',
 'harika',
 'haydi',
 'hemen',
 'her',
 'herkes',
 'hey',
 'hiç',
 'hiçbir',
 'iki',
 'ile',
 'ilk',
 'ister',
 'istiyorsun',
 'istiyorum',
 'iyi',
 'izin',
 'içinde',
 'iş',
 'işte',
 'kendi',
 'ki',
 'kimse',
 'kötü',
 'küçük',
 'kız',
 'lazım',
 'lütfen',
 'merhaba',
 'misin',
 'mu',
 'musun',
 'mü',
 'mısın',
 'nasıl',
 'neden',
 

In [41]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [42]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_30_The_Most_File_28_Word_Twogram_10_Youtube_0.6s_Timeshift_Result.xlsx']

In [58]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [59]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [ ]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [ ]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

In [ ]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

In [ ]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [ ]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

In [ ]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num

In [ ]:
df_word_group_time_loc_result2 = word_group_time_loc(df_search_result2, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2.start_time = df_word_group_time_loc_result2.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result2.end_time = df_word_group_time_loc_result2.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result2['video_id'].map(str)+"&t="+df_word_group_time_loc_result2['start_time'].map(str)+"s"
df_word_group_time_loc_result2

In [ ]:
search_result_num2 = df_word_group_time_loc_result2["search_string"].nunique()
search_result_num2

In [ ]:
df_word_group_time_loc_result2.to_excel(f"{lang_folder.capitalize()}_{search_result_num2}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result2.xlsx", index=False) 